# Cifar image classification using TF 2.0 Keras 
# Classificação Multi-Classe

In [10]:
import tensorflow as tf
from tensorflow import keras
import datetime as dt

In [11]:
tf.__version__

'2.0.0-beta1'

## Dividir os dados em teste e treino

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.math.divide(tf.cast(x, tf.float32), 255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_dataset = train_dataset.repeat()

170500096/170498071 [==============================] - 52s 0us/step


In [3]:
# test dataset
valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1000).shuffle(10000)

In [4]:
valid_dataset = valid_dataset.map(lambda x, y: (tf.math.divide(tf.cast(x, tf.float32),255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))


In [5]:
valid_dataset = valid_dataset.repeat()

# Criar o Modelo Convolutional Neural Network

In [6]:
class CIFAR10Model(keras.Model):
    def __init__(self):
        super(CIFAR10Model, self).__init__(name='cifar_cnn')
        self.conv1 = keras.layers.Conv2D(64, 5,activation=tf.nn.relu)                             
                                         
        self.max_pool2d = keras.layers.MaxPooling2D((3, 3), (2, 2))
        self.max_norm = keras.layers.BatchNormalization()
        self.conv2 = keras.layers.Conv2D(64, 5,activation=tf.nn.relu)                 
                                        
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(750, activation=tf.nn.relu)
                                      
        self.fc2 = keras.layers.Dense(10)
        self.softmax = keras.layers.Softmax()

    def call(self, x):
        x = self.max_pool2d(self.conv1(x))
        x = self.max_norm(x)
        x = self.max_pool2d(self.conv2(x))
        x = self.max_norm(x)
        x = self.flatten(x)
        x = self.fc2(x)
        return self.softmax(x)

# Compilar e Treinar o Modelo

In [7]:
model = CIFAR10Model()

In [8]:

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(train_dataset, epochs=2, steps_per_epoch=100,
          validation_data=valid_dataset,
          validation_steps=3)


Epoch 1/2


W0619 12:22:37.210554  4620 deprecation.py:323] From C:\Users\Bruno\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100/100 [==============================] - 28s 276ms/step - loss: 1.9351 - accuracy: 0.2757 - val_loss: 4.6473 - val_accuracy: 0.0957
Epoch 2/2
100/100 [==============================] - 15s 147ms/step - loss: 1.6245 - accuracy: 0.4111 - val_loss: 6.3780 - val_accuracy: 0.1063
